<a href="https://colab.research.google.com/github/ridhimagarg/PyTorchBook/blob/main/Chapter05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from PIL import Image

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Normalize

# from data_generation.image_classification import generate_dataset ## for directory run
from image_classification import generate_dataset
from helpers import index_splitter, make_balanced_sampler
# from stepbystep.v1 import StepByStep ## for directory run
from v1 import StepByStep

In [ ]:
single = np.array(
  [[[[5, 0, 8, 7, 8, 1],
  [1, 9, 5, 0, 7, 7],
  [6, 0, 2, 4, 6, 6],
  [9, 7, 6, 6, 8, 4],
  [8, 3, 8, 5, 1, 3],
  [7, 2, 7, 0, 1, 0]]]]
)
single.shape

(1, 1, 6, 6)

In [ ]:
identity = np.array(
  [[[[0, 0, 0],
  [0, 1, 0],
  [0, 0, 0]]]]
)
identity.shape


(1, 1, 3, 3)

In [ ]:
region = single[:, :, 0:3, 0:3] 
filtered_region = region * identity
total = filtered_region.sum()

In [ ]:
new_region = single[:, :, 0:3, (0+1):(3+1)]
new_filtered_region = new_region * identity
total = new_filtered_region.sum()

In [ ]:
last_horizontal_region = single[:, :, 0:3, (0+4):(3+4)]
last_horizontal_region * identity

ValueError: ignored

## Convolving in Pytorch

In [ ]:
image = torch.as_tensor(single).float()
kernel_identity = torch.as_tensor(identity).float()

In [ ]:
## Two ways of defining the convolutional -> Functional, Module

convolved = F.conv2d(image, kernel_identity, stride=1)
convolved

tensor([[[[9., 5., 0., 7.],
          [0., 2., 4., 6.],
          [7., 6., 6., 8.],
          [3., 8., 5., 1.]]]])

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1)
conv(image)

tensor([[[[-4.9333, -7.2610, -5.6285, -6.6524],
          [-6.9281, -5.7925, -5.3011, -5.8231],
          [-8.7037, -5.3121, -5.1267, -7.4925],
          [-8.5701, -5.6458, -7.8008, -4.1924]]]],
       grad_fn=<ThnnConv2DBackward0>)

In [ ]:
## for multiple filters

conv_multiple = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1)
conv_multiple.weight

Parameter containing:
tensor([[[[ 0.1172,  0.2258,  0.1416],
          [-0.1706,  0.0261, -0.0372],
          [ 0.0198,  0.1870, -0.1678]]],


        [[[-0.0163,  0.0841, -0.0644],
          [ 0.2509, -0.0808, -0.2158],
          [ 0.0348,  0.1144, -0.2342]]]], requires_grad=True)